In [1]:
import os
import boto3

#1. s3에서 결과 파일 다운받기 

In [19]:
BUCKET_NAME = ''
s3_client = boto3.client('s3')

In [47]:
model_info={"alexnet" : 233.1,
            "efficientnet_b0":20.5,
            "inception_v3":104,
            "mnasnet": 17,
            "mobilenet_v2":13.6,
            "resnet18":44.7,
            "resnet50":97.8,
            "shufflenet":8.9,
            "squeezenet":4.8,
            "vgg16":527.8}
model_name = list(model_info.keys())
hardware_list=["arm","intel"]
optimize_list=["base","onnx","tvm"]
lambda_memory_list=[512,1024,2048,4096,8192,10240]

In [41]:
lambda_memory=10240
batchsize=1

In [42]:
for hardware in hardware_list:
    for optimize in optimize_list:
        for model in model_name:
            try:
                os.makedirs(os.path.dirname(f'./exp1/{lambda_memory}/{optimize}_{hardware}/'), exist_ok=True)
                s3_client.download_file(BUCKET_NAME, 
                                    f'results/{optimize}/{hardware}/{model}_{model_info[model]}_{batchsize}_{lambda_memory}.json',
                                    f'./exp1/{lambda_memory}/{optimize}_{hardware}/{model}_{model_info[model]}_{batchsize}.json')
            except:
                print(f"Not Exist in S3 :'results/{optimize}/{hardware}/{model}_{model_info[model]}_{batchsize}_{lambda_memory}.json', ")

# json to pandas

In [48]:
import json
import pandas as pd

total=pd.DataFrame()
for lambda_memory in lambda_memory_list:
    for hardware in hardware_list:
        for optimize in optimize_list:
            for model in model_name:
                try:
                    with open(f'./exp1/{lambda_memory}/{optimize}_{hardware}/{model}_{model_info[model]}_{batchsize}.json') as f:
                        js = json.loads(f.read()) ## json 라이브러리 이용
                        df = pd.DataFrame([js])
                    total = pd.concat([total,df])
                except:
                    continue

In [49]:
total = total.reset_index(drop=True)

In [50]:
total

,model_name,model_size,hardware,framework,optimizer,lambda_memory,batchsize,convert_time,inference_time,request_id,log_group_name,max_memory_used
0,efficientnet_b0,20.5,arm,torch,base,512,1,0.000000,18.068951,8d32b466-9c36-4a6e-a2cd-9d3a15760ed0,/aws/lambda/serving_arm_torch_512,363
1,inception_v3,104.0,arm,torch,base,512,1,0.000000,29.117535,414a05ee-f771-478e-af87-8737393fe22a,/aws/lambda/serving_arm_torch_512,512
2,mnasnet,17.0,arm,torch,base,512,1,0.000000,11.373008,bbea78b9-5e1f-49c7-a5ba-02f70b54252a,/aws/lambda/serving_arm_torch_512,313
3,mobilenet_v2,13.6,arm,torch,base,512,1,0.000000,10.829113,8d1d7ea0-c56a-4fa7-bd9c-049aa0047881,/aws/lambda/serving_arm_torch_512,337
4,resnet18,44.7,arm,torch,base,512,1,0.000000,10.346255,4d955bee-afbf-4900-9ab5-781a3c2b0cfc,/aws/lambda/serving_arm_torch_512,387
...,...,...,...,...,...,...,...,...,...,...,...,...
337,resnet18,44.7,intel,torch,tvm,10240,1,25.381404,1.622838,3ce24d04-90a9-47b6-b6d9-9c97b612c020,/aws/lambda/serving_intel_tvm_10240,408
338,resnet50,97.8,intel,torch,tvm,10240,1,56.477069,3.536385,0d58f57e-284d-4743-bdc2-d4d253d41a6e,/aws/lambda/serving_intel_tvm_10240,624
339,shufflenet,8.9,intel,torch,tvm,10240,1,57.644670,0.575265,d6e3ed25-4ed7-454d-8283-99483e3c36f0,/aws/lambda/serving_intel_tvm_10240,259
340,squeezenet,4.8,intel,torch,tvm,10240,1,13.889928,0.626686,b81a8fbd-7535-44de-b353-e4e9ce57967e,/aws/lambda/serving_intel_tvm_10240,240


In [51]:
total.to_csv("./total_df.csv")